In [34]:
import pandas as pd
import numpy as np
import requests
import bokeh.plotting as bpl
from bokeh.io import export_png

In [35]:
bpl.output_notebook()

Loading BokehJS ...

In [36]:
response = requests.get("https://wuhan-coronavirus-api.laeyoung.endpoint.ainize.ai/jhu-edu/timeseries")

In [37]:
response.status_code

200

In [38]:
salida=response.json()

In [39]:
data = []
for ele in salida:
    pais = ""
    region = ""
    iso2 = ""
    iso3 = ""
    if ('countryregion' in ele.keys()):
        pais = ele['countryregion']
    if ('provincestate' in ele.keys()):
        region = ele['provincestate']
    if ('countrycode' in ele.keys()):
        if ('iso2' in ele['countrycode'].keys()):
            iso2 = ele['countrycode']['iso2']
        if ('iso3' in ele['countrycode'].keys()):
            iso3 = ele['countrycode']['iso3']
    if ('timeseries' in ele.keys()):
        serie = ele['timeseries']
        for time in serie:
            confirmados = None
            muertes = None
            recovered = None
            if ('confirmed' in serie[time].keys()):
                confirmados = serie[time]['confirmed']
            if ('deaths' in serie[time].keys()):
                muertes = serie[time]['deaths']
            if ('confirmed' in serie[time].keys()):
                recovered = serie[time]['recovered']
            data.append([pais,region,iso2,iso3,time,confirmados,muertes,recovered])

In [40]:
df = pd.DataFrame(data,columns=["Country","Province","iso2","iso3","date","confirmed","deaths","recovered"])

In [41]:
df["Fecha"] = pd.to_datetime(df["date"],format="%m/%d/%y")

In [42]:
df[:3]

,Country,Province,iso2,iso3,date,confirmed,deaths,recovered,Fecha
0,Afghanistan,,AF,AFG,1/22/20,0.0,0.0,0.0,2020-01-22
1,Afghanistan,,AF,AFG,1/23/20,0.0,0.0,0.0,2020-01-23
2,Afghanistan,,AF,AFG,1/24/20,0.0,0.0,0.0,2020-01-24


In [43]:
df["Fecha"].max()

Timestamp('2020-05-13 00:00:00')

In [44]:
df["Country"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Ira

In [89]:
pais_seleccionado = "Colombia"
datos_filt = df[(df["Country"]==pais_seleccionado)&(df["confirmed"]>0)]

In [90]:
datos_plot = datos_filt.groupby(["Fecha"])[["confirmed","deaths","recovered"]].sum()

In [91]:
datos_plot[:3]

,confirmed,deaths,recovered
Fecha,,,
2020-03-06,1.0,0.0,0.0
2020-03-07,1.0,0.0,0.0
2020-03-08,1.0,0.0,0.0


In [92]:
datos_plot["deaths"]

Fecha
2020-03-06      0.0
2020-03-07      0.0
2020-03-08      0.0
2020-03-09      0.0
2020-03-10      0.0
              ...  
2020-05-09    445.0
2020-05-10    463.0
2020-05-11    479.0
2020-05-12    493.0
2020-05-13    509.0
Name: deaths, Length: 69, dtype: float64

In [93]:
datos_plot["dias_transcurridos"] = pd.Series(range(len(datos_plot)),index=datos_plot.index)

In [94]:
datos_plot["activos"] = datos_plot["confirmed"]-datos_plot["deaths"]-datos_plot["recovered"]

In [95]:
datos_plot[-3:]

,confirmed,deaths,recovered,dias_transcurridos,activos
Fecha,,,,,
2020-05-11,11613.0,479.0,2825.0,66,8309.0
2020-05-12,12272.0,493.0,2971.0,67,8808.0
2020-05-13,12930.0,509.0,3133.0,68,9288.0


In [96]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title = pais_seleccionado)

In [97]:
p.vbar(x=datos_plot.index,top=datos_plot["activos"],bottom=1,width=(datos_plot.index[1]-datos_plot.index[0])*0.8, color="red",fill_alpha=0.7,legend_label="Casos activos")
p.vbar(x=datos_plot.index,top=datos_plot["activos"]+datos_plot["deaths"],bottom=datos_plot["activos"],width=(datos_plot.index[1]-datos_plot.index[0])*0.8, color="#444549",fill_alpha=0.7,legend_label="Decesos")
p.vbar(x=datos_plot.index,top=datos_plot["activos"]+datos_plot["deaths"]+datos_plot["recovered"],bottom=datos_plot["activos"]+datos_plot["deaths"],width=(datos_plot.index[1]-datos_plot.index[0])*0.8, color="#2EC750",fill_alpha=0.7,legend_label="Casos recuperados")
p.line(x=datos_plot.index,y=datos_plot["confirmed"],color="#7B0025",line_width=3,legend_label="Casos confirmados")

GlyphRenderer(id='3464', ...)

In [98]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [99]:
bpl.show(p)